In [1]:
#import tensorflow as tf
#gpus = tf.config.experimental.list_physical_devices('GPU')
#if gpus:
#    try:
#        for gpu in gpus:
#            tf.config.experimental.set_memory_growth(gpu, True)

#    except RuntimeError as e:
#        print(e)

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

C:\Users\James_Dev_Account\anaconda3\envs\honours-keras-2.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\James_Dev_Account\anaconda3\envs\honours-keras-2.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\James_Dev_Account\anaconda3\envs\honours-keras-2.1.4\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

KeyboardInterrupt: 

In [ ]:
"""

This is for morphological classification of galaxies by CNN,
By Kenji Bekki, on 2017/11/15
Revised on 2020/2/14 (Nair & Abraham 2010)
For test only.

"""


import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from keras.models import model_from_json
import keras.callbacks
import numpy as np
#import keras.backend.tensorflow_backend as KTF
#import tensorflow as tf
import os.path


### Original values
#batch_size = 128
#num_classes = 10
#epochs = 12
#batch_size = 200

num_classes = 2
num_classes0 = 2

#epochs = 1
#nb_epoch=epochs

n_mesh=50

img_rows, img_cols = n_mesh, n_mesh
n_mesh2=n_mesh*n_mesh-1
n_mesh3=n_mesh*n_mesh

#input_shape = (img_rows, img_cols, 1)

# For output the galaxy classification results
f1=open('test27d.out','w')

nmodel=1000

In [ ]:
# This is for simlation data sets
#GD_W_DISC_PATH = "galaxy_data_including_disc/m1.dir/"
#GD_WO_DISC_PATH = "galaxy_data_without_disc/m1.dir/"

#GD_FV = "2dfv_test_split.dat"
#DATA_PART_NMODEL = int(nmodel/2)


#def load_data(path, filename):
#    return np.genfromtxt(os.path.join(path, filename), autostrip=True, max_rows=nmodel*n_mesh3)

#x_test = load_data("", GD_FV)

#ntest = int(x_test.size / n_mesh3)

#x_test = x_test.reshape(ntest, img_rows, img_cols, 1)

In [ ]:
def old():
    # This is for simlation data sets
    GD_W_DISC_PATH = "../generated_data/data3/"
    GD_WO_DISC_PATH = "../generated_data/data4/"

    GD_FN = "2dft.dat"
    DATA_PART_NMODEL = int(nmodel/2)


    def load_data(path, filename):
        return np.genfromtxt(os.path.join(path, filename), autostrip=True, skip_header=DATA_PART_NMODEL*n_mesh3, max_rows=DATA_PART_NMODEL*n_mesh3)

    x_dataset_wd = load_data(GD_W_DISC_PATH, GD_FN)
    y_dataset_wd = np.ones((DATA_PART_NMODEL, 1), np.int)

    x_dataset_wod = load_data(GD_WO_DISC_PATH, GD_FN)
    y_dataset_wod = np.zeros((DATA_PART_NMODEL, 1), np.int)

    x_dataset = np.append(x_dataset_wd, x_dataset_wod, axis=0)
    y_dataset = np.append(y_dataset_wd, y_dataset_wod, axis=0)

    np.savetxt("2dftesting.dat", x_dataset, '%1.8f')
    np.savetxt("2dftestingn.dat", y_dataset, '%d')

    x_dataset = x_dataset.reshape(nmodel, img_rows, img_cols, 1)
    print(x_dataset.shape)
    print(y_dataset.shape)

    ntest = int(nmodel)
    x_test = x_dataset

In [ ]:

%run ../data_preparation/FILE_IO_FUNCTIONS.ipynb
nmodel_part = int(nmodel/2)

x_dataset_wd = load_data(data="DATA3", count=nmodel_part, skip=nmodel_part)
x_dataset_wod = load_data(data="DATA4", count=nmodel_part, skip=nmodel_part)

y_dataset_wd = np.ones((x_dataset_wd.shape[0], 1), np.int)
y_dataset_wod = np.zeros((x_dataset_wod.shape[0], 1), np.int)

x_dataset = np.append(x_dataset_wd, x_dataset_wod, axis=0)
y_dataset = np.append(y_dataset_wd, y_dataset_wod, axis=0)

%run ../data_preparation/PRE_PROCESSING.ipynb
add_gaussian_noise(x_dataset)
add_point_sources(x_dataset)

print(x_dataset.shape)
print(y_dataset.shape)

In [ ]:
load_nair_data()

%run ../data_preparation/PRE_PROCESSING.ipynb
add_gaussian_noise(x_dataset)
add_point_sources(x_dataset)

print(x_dataset.shape)
print(y_dataset.shape)
ntest = int(nmodel)
x_test = x_dataset

In [ ]:

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

#y_vec=np.zeros(3)
#y_vec=np.zeros(num_classes)
y_vec=np.zeros(num_classes)
#print(y_vec)

y_pred=loaded_model.predict(x_test)
print(y_pred[:ntest])

f1.write( str(ntest) + "\n" )

for i in range(ntest):
#  for j in range(num_classes0):
  for j in range(num_classes):
    y_vec[j]=y_pred[i,j]
#  print(y_vec)
#    print(j)
  y_type=np.argmax(y_vec)
#  y_type=y_type+1
  prob=y_vec[y_type]
  print('i=',i,'G-type=',y_type,'P',prob)
#  Original  type-1 is output
  f1.write( str(y_type) + ' ' + str(y_vec[0]) + ' '+
  str(y_vec[1]) + "\n" )
#  f1.write( str(y_type) +
#   "\n" )
f1.flush()

#loaded_model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=keras.optimizers.Adadelta(),
#              metrics=['accuracy'])




In [ ]:
from sklearn import metrics

true_data = y_dataset
predicted_data = np.loadtxt('test27d.out', skiprows=1, usecols=(0))
#print(y_pred)


metrics.ConfusionMatrixDisplay(metrics.confusion_matrix(true_data, predicted_data)).plot(cmap='Blues')

In [ ]:
import pandas as pd
df = pd.DataFrame(true_data)
df.value_counts()